In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# self write tools
from tools import LABEL_ENCODER
from models import RNN
from dataset import CnnDataset, TestDataset, CREATE_CNN_TRAIN_ITERATOR, CREATE_CNN_VALID_ITERATOR, words


<IPython.core.display.Javascript object>

      id  category  gender  baseColour  season  usage  \
0  47381        21       4           2       3      0   
1  13218        24       2          45       2      2   
2  41776        24       4          31       0      0   
3  56271        12       2          25       2      0   
4  27361         9       2          32       1      0   

                              noisyTextDescription  \
0                              HM Women Blue Flats   
1                     Arrow Men Yellow Danny Shirt   
2  Chhota Bheem Kids Qsm I Love to red Pink Tshirt   
3           Jockey Textura Men Pack Of 2 Duet 1010   
4                         Wild stone Men Juice Deo   

                                       description  description_length  \
0                              HM Women Blue Flats                   4   
1                     Arrow Men Yellow Danny Shirt                   5   
2  Chhota Bheem Kids Qsm I Love to red Pink Tshirt                  10   
3           Jockey Textura Men Pack 

# Check GPU

In [2]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
torch.backends.cudnn.benchmark = True
print("Using {}: {}".format(device, torch.cuda.get_device_name(0)))

Using cuda: GeForce RTX 3080


In [3]:
BATCH_SIZE = 256



In [4]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    best_valid_loss = float('inf')
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        correct = 0

        for batch in train_dl:

            _, (x, s), features, y = batch
            x = x.long()
            y = y.long()



            y_pred = model(x, s)

            top_pred = y_pred.argmax(1, keepdim = True)
            correct += top_pred.eq(y.view_as(top_pred)).sum()
            

            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]



        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)

        # save best model
        if val_loss < best_valid_loss:
            best_valid_loss = val_loss
            torch.save(model.state_dict(), './output/rnn-model.pt')

    
        print('epoch:', i, ', train acc: ', correct / total)
        print("train loss %.3f, val loss %.3f, val accuracy %.2f%%, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))
        if correct / total > 0.99:
            break

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    # for x, y, l in valid_dl:
    for _, (x, l), features, y in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]



    return sum_loss/total, correct/total * 100, sum_rmse/total

In [5]:
vocab_size = len(words)

train_dl = CREATE_CNN_TRAIN_ITERATOR(None, BATCH_SIZE)
val_dl = CREATE_CNN_VALID_ITERATOR(BATCH_SIZE)



In [6]:
model = RNN()

In [7]:
train_model(model, epochs=100, lr=0.001)

epoch: 0 , train acc:  tensor(0.4208)
train loss 2.581, val loss 1.735, val accuracy 53.65%, and val rmse 9.316
epoch: 1 , train acc:  tensor(0.6084)
train loss 1.516, val loss 1.200, val accuracy 70.60%, and val rmse 7.146
epoch: 2 , train acc:  tensor(0.7216)
train loss 1.151, val loss 0.947, val accuracy 76.24%, and val rmse 6.372
epoch: 3 , train acc:  tensor(0.7661)
train loss 0.946, val loss 0.812, val accuracy 78.99%, and val rmse 5.993
epoch: 4 , train acc:  tensor(0.7926)
train loss 0.827, val loss 0.739, val accuracy 80.37%, and val rmse 5.805
epoch: 5 , train acc:  tensor(0.8134)
train loss 0.744, val loss 0.679, val accuracy 82.11%, and val rmse 5.548
epoch: 6 , train acc:  tensor(0.8279)
train loss 0.679, val loss 0.638, val accuracy 82.85%, and val rmse 5.434
epoch: 7 , train acc:  tensor(0.8423)
train loss 0.622, val loss 0.611, val accuracy 83.68%, and val rmse 5.413
epoch: 8 , train acc:  tensor(0.8520)
train loss 0.573, val loss 0.585, val accuracy 84.51%, and val rms